In [1]:
import pandas as pd

data = pd.read_pickle('df_for_WebApp.pkl')
data

,Summary,EmbeddingsSummary,Description,Comments
0,plugin config-import not found for jfrog Artif...,"[0.0066620787, 0.12316594, -0.0076982556, 0.08...",I am following the steps from [https://docs.jf...,[I am running the command provided by the docu...
1,RTDEV-5069 / mirroring https://asset-packagist...,"[-0.08578057, 0.063168734, -0.0045067403, 0.17...",We are running Artifactory 7.63.14 and have pr...,"[{quote}RTDEV-5069 Fixed an issue whereby, Art..."
2,Docker - Repository Default URL,"[-0.0026910885, 0.054876674, -0.013277466, -0....",Similar to Issue: [https://jfrog.atlassian.net...,[This is the current value I would like to cha...
3,getting manifest unknown error pulling image,"[-0.10222563, -0.027658874, -0.005382626, 0.11...",I am getting the following error when pulling ...,"[{\n ""schemaVersion"": 2,\n ""mediaType"": ""a..."
4,Caught exception while saving incoming stream ...,"[-0.0813854, 0.14918321, -0.007328004, 0.02076...",We have recently upgrade Artifactory to 7.63.1...,"[""PUT ‘REPO_PATH/s.zip’ HTTP/1.1"" 400 0 ""-"" ""p..."
...,...,...,...,...
9414,Gradle plugin should use full compilation clas...,"[-0.038508527, 0.23863803, -0.008430785, 0.200...",[This Spring Boot 3 + Hibernate 6 sample|https...,[Going to need a little more information. Wha...
9415,Illegal Update-Statement generated,"[-0.053460695, 0.12169801, -0.014299179, 0.006...",The following Entities are given\n\n{noformat}...,[This is due to your use of {{not-found=”ignor...
9416,Trying to accomplish Insert bulk query with hi...,"[0.0037901562, 0.03531771, -0.0055623963, 0.06...",We are working on a spring boot api which uses...,"[, nan, nan, nan, nan, nan, nan, nan, nan, nan..."
9417,Emulate lt/gt/le/ge tuple-comparisons when uns...,"[-0.041004553, -0.06743485, -0.008913418, 0.13...",Execution of the {{ ./gradlew clean test -Pdb...,[Thanks for the report. I see that HANA only s...


In [4]:
type(data['EmbeddingsSummary'][0])


numpy.ndarray

In [15]:
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity


model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")


def col_embed(df):
    embeddings = []

    # Iterate through each row of the column
    for i in df:
        # Encode the sentence using the SBERT model
        embedding = model.encode(i)
        # Append the embedding to the list
        embeddings.append(embedding)

    return embeddings

def sentence_embed(sentence):
    embedding = model.encode(sentence)
    return embedding


def calculate_similarity(input_embedding, embeddings):
    # input_embedding = model.encode(input_sentence)

    similarities = []
    for embedding in embeddings.tolist():
        similarity = cosine_similarity(input_embedding.reshape(1, -1), embedding.reshape(1, -1))[0][0]
        similarities.append(similarity)

    return similarities


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9419 entries, 0 to 9418
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Summary                9419 non-null   object
 1   EmbeddingsSummary      9419 non-null   object
 2   Description            9418 non-null   object
 3   EmbeddingsDescription  9419 non-null   object
 4   Comments               9419 non-null   object
dtypes: object(5)
memory usage: 368.1+ KB


In [10]:
data.describe()

,Summary,EmbeddingsSummary,Description,EmbeddingsDescription,Comments
count,9419,9419,9418,9419,9419
unique,9419,9417,9418,9411,8470
top,plugin config-import not found for jfrog Artif...,[-1.46329835e-01 1.25124604e-01 -9.10426583e-...,I am following the steps from [https://docs.jf...,[ 1.28439593e-03 -1.11920826e-01 -7.68679567e-...,"['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'na..."
freq,1,2,1,2,577


In [16]:
x= 'What is the best way to get a job in data science?'
x1 = sentence_embed(x)
sum_similarity = calculate_similarity(x1, data['EmbeddingsSummary'])

AttributeError: 'str' object has no attribute 'reshape'

In [2]:
type(data['EmbeddingsSummary'][0])

str

In [ ]:
# Store summaries with similarity > 0.7 and their scores in a list
similar_summaries_and_scores = []
for summary, similarity in zip(data["Summary"], sum_similarity):
  if similarity > 0.5:
    similar_summaries_and_scores.append((summary, similarity))

# Sort similar summaries by similarity score (highest first)
sorted_similar_summaries_and_scores = sorted(similar_summaries_and_scores, key=lambda x: x[1], reverse=True)

# Print top 5 similar sentences and their scores
for i, (summary, similarity) in enumerate(sorted_similar_summaries_and_scores):
  if len(sum_similarity) == 0:
    print('No similar tickets found.')
  if i == 5:
    break  # Print only the top 5
  print(f"Similar summary: {summary} \n , Similarity: {similarity:.4f} \n")